KHAI BÁO THƯ VIỆN



In [1]:

import matplotlib.pyplot as plt
import math
import numpy as np 
import pandas as pd 
import io 
import requests
import datetime as dt


CÁC CÔNG THỨC CHUYỂN ĐỔI


In [2]:
def Calculate_Saturation_vapour_pressure(t):
    return 610.78*math.exp((t/(t+238.3))*17.2694)
def Calculate_vapour_pressure_From_Humdity(RH,t):
    return (RH)*610.78*math.exp((t/(t+238.3))*17.2694)
def ConvertAirFrom_PPM_To_mgDivm3(x):
    return x*(1/0.554)


KHAI BÁO PHẦN THAM SỐ CÓ SẴN


In [3]:
M_water=18 #kg (kmol)^-1
h_Air=3.8 #m
R=8314 #J*(kmol)^-1*K^-1
h_Gh=4.2 #m
Eta_Global_Air=0.1 #no unit
Eta_Global_PAR=0.5 #no unit
Tau_Cov_PAR=0.534311577 #no unit
Eta_Global_NIR=0.5 #no unit
Tau_Cov_NIR=0.534311577 #no unit
c_evap1=4.3 #W*m^-2
c_evap2=0.54 #W*m^-2
s_r_s=-1 #m/W^2
R_Can_SP= 5 #W*m^-2
night_c_evap3= 1.1*(10**(-11)) #ppm^-2
day_c_evap3= 6.1*(10**(-7)) #ppm^-2
night_c_evap4= 5.2*(10**(-6)) #Pa^-2
day_c_evap4= 4.3*(10**(-6)) #Pa^-2
eta_mg_ppm= 0.554 #ppm*mg^-1*m^3
r_s_min= 82 #s/m
rho_Air= 1.2 #kg/m^3
c_p_Air= 1000 #J/(K*kg)
LAI=3 #m^2 {leaf} m^-2
deltaH=2.45*(10**6) #J/(kg{Water})
gamma=65.8 #Pa*K^-1
r_b= 275 #s/m
U_Pad= 1 #no unit
phi_Pad=35.6#m^3/s 
A_Flr=14000 #m^2
eta_Pad=0 #no unit
x_Pad=0 #kg{water} kg^(-1){air}
x_Out=0 #kg{water} kg^(-1){air}
U_Fog=1 #no unit
phi_Fog=0 #kg {water}/s
eta_HeatVap= 4.43*(10**(-8)) #Kg vapour/ J
U_Blow=1 #no unit
P_Blow=500 #W
U_ThScr=0.75 #no unit
K_ThScr=0.05*(10**(-3)) #m*K^(-2/3)*s^(-1)
Delta_T_AirTop=1 #K
g=9.81 #(m/s^2)
RO_Mean=1.2 #kg*m^(-3) 
Delta_RO=0.0004837 #? #kg*m^(-3)
Zeta_InsScr=1 #no unit
C_leakage=1*(10**(-4))
C_d=0.75 #no unit
U_Roof=0.05 #no unit
U_Side=0.05 #no unit
A_Roof=1400 #m^2
A_Side=1400 #m^2
h_Roof=0.68 #m
h_SideRoof=1.9 #m
Delta_T_AirOut=-5 #k
T_mean=298 #K
C_w=0.09 #no unit
Eta_RoofThr=0.9 #no unit
Eta_SideThr=0.9 #no unit
U_MechCool=1 #no unit
COP_MechCool=0 #no unit
P_MechCool=0 #no unit
A_Cov=18000 #m^2
c_HECin=1.86 #W*m^-2*K^-1
U_VentForced=1
Phi_VentForced=0

CÁC PHƯƠNG TRÌNH TÍNH TỪNG BIỂU THỨC

DÙNG BIẾN TOÀN CỤC ĐỂ LƯU GIÁ TRỊ VÀ CÁC GIÁ TRỊ TRẢ VỀ Ở MỖI BIỂU THỨC ĐỂ ÁP DỤNG CHO CÔNG THỨC TIẾP THEO

In [4]:
#Phương trình tính cap_VP_Air
def def_cap_VP_Air(M_water, h_Air, R, T_Air):
    return ((M_water*h_Air)/(R*(T_Air+273.15)))

#Phương trình tính cap_VP_Top
def def_cap_VP_Top(M_water, h_Gh, R, T_Top):
    return ((M_water*h_Gh)/(R*(T_Top+273.15)))

#Phương trình tính R_Can
def def_R_Can(Eta_Global_Air,I_Global,Eta_Global_PAR,Tau_Cov_PAR,Eta_Global_NIR,Tau_Cov_NIR):
    return (1-Eta_Global_Air)*I_Global*(Eta_Global_PAR*Tau_Cov_PAR+Eta_Global_NIR*Tau_Cov_NIR)

#Phương trình tính rf_R_can
def def_rf_R_can(R_Can, c_evap1, c_evap2):
    return (R_Can + c_evap1)/(R_Can + c_evap2)

#Phương trình tính S_r_S
def def_S_r_S(s_r_s, R_Can, R_Can_SP):
    return 1/(1+math.exp(s_r_s*(R_Can-R_Can_SP)))

#Phương trình tính c_evap3
def def_c_evap3(night_c_evap3, S_r_S,day_c_evap3):
    return night_c_evap3*(1-S_r_S)+day_c_evap3*S_r_S

#Phương trình tính c_evap4
def def_c_evap4(night_c_evap4, S_r_S,day_c_evap4):
    return night_c_evap4*(1-S_r_S)+day_c_evap4*S_r_S

#Phương trình tính rf_CO2_Air_ppm
def def_rf_CO2_Air_ppm(c_evap3, eta_mg_ppm, CO2_Air):
    return ( 1+ c_evap3*((eta_mg_ppm*CO2_Air-200)**2) )

#Phương trình tính rf_VP_Can_VP_Air
def def_rf_VP_Can_VP_Air(c_evap4, VP_Can, VP_Air):
    return (1+c_evap4*(VP_Can-VP_Air)**(2))

#Phương trình tính r_s
def def_r_s(r_s_min, rf_R_Can, rf_CO2_Air_ppm, rf_VP_Can_VP_Air):
    return (r_s_min*rf_R_Can*rf_CO2_Air_ppm*rf_VP_Can_VP_Air)

#Phương trình tính VEC_CanAir
def def_VEC_CanAir(rho_Air, c_p_Air, LAI, deltaH, gamma, r_b, r_s):
    return (2*rho_Air*c_p_Air*LAI)/(deltaH*gamma*(r_b+r_s))

#Phương trình tính MV_CanAir
def def_MV_CanAir(VEC_CanAir, VP_Can, VP_Air):
    return (VEC_CanAir*(VP_Can-VP_Air))

#Phương trình tính f_Pad
def def_f_Pad(U_Pad,phi_Pad,A_Flr):
    return U_Pad*phi_Pad/A_Flr

#Phương trình tính MV_PadAir
def def_MV_PadAir(rho_Air, f_Pad, eta_Pad, x_Pad, x_Out):
    return (rho_Air*f_Pad*(eta_Pad*(x_Pad-x_Out)+x_Out))

#Phương trình tính MV_AirOut_Pad
def def_MV_AirOut_Pad(f_Pad, M_water, VP_Air, R, T_Air):
    return (f_Pad*(M_water/R)*(VP_Air/(T_Air+273.15)))

#Phương trình tính MV_FogAir
def def_MV_FogAir(U_Fog, pi_Fog, A_Flr):
    return (U_Fog*phi_Fog/A_Flr)

#Phương trình tính MV_BlowAir
def def_MV_BlowAir(eta_HeatVap, U_Blow, P_Blow, A_Flr):
    return ((eta_HeatVap*U_Blow*P_Blow)/A_Flr)

#Phương trình tính HEC_AirThScr
def def_HEC_AirThScr(U_ThScr, T_Air, T_ThScr):
    return (1.7*U_ThScr*abs(T_Air-T_ThScr)**(0.33))

#Phương trình tính MV_AirThScr
def def_MV_AirThScr(HEC_AirThScr, VP_Air, VP_ThScr):
    if (VP_Air < VP_ThScr):
        return 0
    else:
        return (6.4*((10**(-9))*HEC_AirThScr*(VP_Air-VP_ThScr)))

#Phương trình tính HEC_AirMech
def def_HEC_AirMech(U_MechCool, COP_MechCool, P_MechCool, A_Flr, T_Air, T_MechCool, deltaH, VP_Air, VP_MechCool):
    return (U_MechCool*COP_MechCool*P_MechCool/A_Flr)/(T_Air-T_MechCool + 6.4*(10**(-9))*deltaH*(VP_Air-VP_MechCool))

#Phương trình tính MV_AirMech
def def_MV_AirMech(HEC_AirMech, VP_Air, VP_MechCool):
    if (VP_Air < VP_MechCool):
        return 0
    else:
        return (6.4*(10**(-9))*HEC_AirMech*(VP_Air-VP_MechCool))

#Phương trình tính HEC_TopCov_in
def def_HEC_TopCov_in(C_HECin, T_Top, T_Cov_in, A_Cov, A_Flr):
    return (c_HECin*((T_Top-T_Cov_in)**(0.33))*A_Cov/A_Flr)

#Phương trình tính MV_TopCov_in
def def_MV_TopCov_in(HEC_TopCov_in, VP_Top, VP_Cov_in):
    if (VP_Top <VP_Cov_in):
        return 0
    else:
        return (6.4*(10**(-9))*HEC_TopCov_in*(VP_Top-VP_Cov_in))
    
#Phương trình tính f_ThScr
def def_f_ThScr(U_ThScr,K_ThScr,Delta_T_AirTop,g,RO_Mean,Delta_RO):
    return U_ThScr*K_ThScr*((Delta_T_AirTop)**(2/3))+(1-U_ThScr)*(((g*(1-U_ThScr)/(2*RO_Mean))*Delta_RO)**(1/2))

#Phương trình tính MV_AirTop
def def_MV_AirTop(M_water, f_ThScr, VP_Air, VP_Top, R, T_Air, T_Top):
    return (M_water/R)*f_ThScr*((VP_Air/(T_Air+273.15))-(VP_Top/(T_Top+273.15)))

#Phương trình tính f_VentSide
def def_f_VentSide(Eta_InsScr,ff_VentSide,f_leakage,U_ThScr,f_VentRoofSide,Eta_Side,Eta_SideThr):
    if (Eta_Side>=Eta_SideThr):
        return Eta_InsScr*ff_VentSide+0.5*f_leakage
    else:
        return Eta_InsScr*(U_ThScr*ff_VentSide+(1-U_ThScr)*f_VentRoofSide*Eta_Side)+0.5*f_leakage

#Phương trình tính Eta_InsScr
def def_Eta_InsScr(Zeta_InsScr):
    return Zeta_InsScr*(2-Zeta_InsScr)

#Phương trình tính f_leakage
def def_f_leakage(v_Wind,C_leakage):
    if (v_Wind<0.25):
        return 0.25*C_leakage
    else:
        return v_Wind*C_leakage
    
#Phương trình tính f_VentRoofSide
def def_f_VentRoofSide(C_d,A_Flr,U_Roof,A_Roof,U_Side,A_Side,g,h_SideRoof,Delta_T_AirOut,T_mean,C_w,v_Wind):
    component1=C_d/A_Flr
    component2=((U_Roof**2)*(U_Side**2)*(A_Roof**2)*(A_Side**2))/((U_Roof**2)*(A_Roof**2)+(U_Side**2)*(A_Side**2))
    component3=(2*g*h_SideRoof*Delta_T_AirOut)/T_mean
    component4=(((U_Roof*A_Roof+U_Side*A_Side)/2)**2)*C_w*(v_Wind**2)
    return component1*((component2*component3+component4)**(1/2))

#Phương trình tính f_VentForced
def def_f_VentForced(Eta_InsScr,U_VentForced,Phi_VentForced,A_Flr):
    return Eta_InsScr*U_VentForced*Phi_VentForced/A_Flr

#Phương trình tính MV_AirOut
def def_MV_AirOut(M_water, f_ventSide, f_ventForced, VP_Air, VP_Out, R, T_Air, T_Out):
    return (M_water/R)*(f_ventSide+f_ventForced)*((VP_Air/(T_Air+273.15))-(VP_Out/(T_Out+273.15)))

#Phương trình tính ff_VentRoof
def def_ff_VentRoof(C_d,U_Roof,A_Roof,A_Flr,g,h_Roof,Delta_T_AirOut,T_mean,C_w,v_Wind):
    component1=C_d*U_Roof*A_Roof/(2*A_Flr)
    component2=(g*h_Roof*Delta_T_AirOut/(2*T_mean))+C_w*(v_Wind**2)
    return component1*(component2**(1/2))

#Phương trình tính f_VentRoof
def def_f_VentRoof(Eta_InsScr,ff_VentRoof,f_leakage,U_ThScr,f_VentRoofSide,Eta_Roof,Eta_RoofThr):
    if (Eta_Roof>=Eta_RoofThr):
        return Eta_InsScr*ff_VentRoof+0.5*f_leakage
    else:
        return Eta_InsScr*(U_ThScr*ff_VentRoof+(1-U_ThScr)*f_VentRoofSide*Eta_Roof)+0.5*f_leakage
    
#Phương trình tính MV_TopOut
def def_MV_TopOut(M_water, f_ventRoof, VP_Top, VP_Out, R, T_Top, T_Out):
    return (M_water/R)*f_ventRoof*((VP_Top/(T_Top+273.15))-(VP_Out/(T_Out+273.15)))

HÀM TÍNH GIÁ TRỊ CALLABLE DX

In [5]:
def dx(cap_VP_Air,cap_VP_Top, MV_CanAir, MV_PadAir, MV_FogAir, MV_AirThScr, MV_AirOut, MV_AirOut_Pad, MV_AirTech, MV_AirTop, MV_TopCov_in, MV_TopOut):
    print("Tính toán cap_VP")
    cap_VP_Air = def_cap_VP_Air(M_water, h_Air, R, T_Air)
    print("phương trình tính cap_VPAir = ", cap_VP_Air)
    cap_VP_Top = def_cap_VP_Top(M_water, h_Gh, R, T_Top)
    print("phương trình tính cap_VP_Top = ", cap_VP_Top)
    print()
    
    print("Tính toán MV_CanAir")
    R_Can=def_R_Can(Eta_Global_Air,I_Global,Eta_Global_PAR,Tau_Cov_PAR,Eta_Global_NIR,Tau_Cov_NIR)
    print("phương trình tính R_Can=",R_Can)
    rf_R_Can =  def_rf_R_can(R_Can, c_evap1, c_evap2)
    print("phương trình tính rf_R_Can=",rf_R_Can)
    S_r_S = def_S_r_S(s_r_s, R_Can, R_Can_SP)
    print("phương trình tính S_r_S=",S_r_S)
    c_evap3 = def_c_evap3(night_c_evap3,S_r_S,day_c_evap3)
    print("phương trình tính c_evap3=",c_evap3)
    c_evap4 = def_c_evap4(night_c_evap4,S_r_S,day_c_evap4)
    print("phương trình tính c_evap4=",c_evap4)
    rf_CO2_Air_ppm = def_rf_CO2_Air_ppm(c_evap3, eta_mg_ppm, CO2_Air)
    print("phương trình tính rf_CO2_Air_ppm=",rf_CO2_Air_ppm)
    rf_VP_Can_VP_Air = def_rf_VP_Can_VP_Air(c_evap4, VP_Can, VP_Air)
    print("phương trình tính rf_VP_VP_Air=",rf_VP_Can_VP_Air)
    r_s = def_r_s(r_s_min, rf_R_Can, rf_CO2_Air_ppm, rf_VP_Can_VP_Air)
    print("phương trình tính r_s=",r_s)
    VEC_CanAir = def_VEC_CanAir(rho_Air, c_p_Air, LAI, deltaH, gamma, r_b, r_s)
    print("phương trình tính VEC_CanAir=",VEC_CanAir)
    MV_CanAir = def_MV_CanAir(VEC_CanAir, VP_Can, VP_Air)
    print("phương trình tính Mv_CanAir = ", MV_CanAir)
    print()
    
    print("Tính toán MV_PadAir")
    f_Pad=def_f_Pad(U_Pad,phi_Pad,A_Flr)
    print("phương trình tính f_Pad=",f_Pad)
    MV_PadAir = def_MV_PadAir(rho_Air, f_Pad, eta_Pad, x_Pad, x_Out)
    print("phương trình tính MV_PadAir = ",MV_PadAir)
    print()
    
    print("Tính toán MV_AirOut_Pad")
    MV_AirOut_Pad = def_MV_AirOut_Pad(f_Pad, M_water, VP_Air, R, T_Air)
    print("phương trình tính MV_AirOut_Pad = ", MV_AirOut_Pad)
    print()
    
    print("Tính toán MV_FogAir")
    MV_FogAir = def_MV_FogAir(U_Fog, phi_Fog, A_Flr)
    print("phương trình tính MV_FogAir = ", MV_FogAir)
    print()
    
    print("Tính toán MV_BlowAir")
    MV_BlowAir = def_MV_BlowAir(eta_HeatVap, U_Blow, P_Blow, A_Flr)
    print("phương trình tính MV_BlowAir = ", MV_BlowAir)
    print()
    
    print("Tính toán MV_AirThScr")
    HEC_AirThScr = def_HEC_AirThScr(U_ThScr, T_Air,T_ThScr)
    print("phương trình tính HEC_AirThScr=",HEC_AirThScr)
    MV_AirThScr = def_MV_AirThScr(HEC_AirThScr, VP_Air, VP_ThScr)
    print("phương trình tính MV_AirThScr = ", MV_AirThScr)
    print()
    
    print("Tính toán MV_AirMech")
    HEC_AirMech = def_HEC_AirMech(U_MechCool, COP_MechCool, P_MechCool, A_Flr, T_Air, T_MechCool, deltaH, VP_Air, VP_MechCool)
    print("phương trình tính HEC_AirMech=",HEC_AirMech)
    MV_AirMech = def_MV_AirMech(HEC_AirMech, VP_Air, VP_MechCool)
    print("phương trình tính MV_AirMech = ", MV_AirMech)
    print()
    
    print("Tính toán MV_TopCov_in")
    HEC_TopCov_in = def_HEC_TopCov_in(c_HECin, T_Top, T_Cov_in, A_Cov, A_Flr)
    print("phương trình tính HEC_TopCov_in = ", HEC_TopCov_in)
    MV_TopCov_in = def_MV_TopCov_in(HEC_TopCov_in, VP_Top, VP_Cov_in)
    print("phương trình tính MV_TopCov_in = ", MV_TopCov_in)
    print()
    
    print("Tính toán MV_AirTop")
    f_ThScr=def_f_ThScr(U_ThScr,K_ThScr,Delta_T_AirTop,g,RO_Mean,Delta_RO)
    print("phương trình tính f_ThScr=",f_ThScr)
    MV_AirTop = def_MV_AirTop(M_water, f_ThScr, VP_Air, VP_Top, R, T_Air, T_Top)
    print("phương trình tính MV_AirTop = ", MV_AirTop)
    print()

    print("Tính toán MV_AirOut")
    Eta_InsScr=def_Eta_InsScr(Zeta_InsScr)
    print("phương trình tính Eta_InsScr=",Eta_InsScr)
    f_leakage=def_f_leakage(v_Wind,C_leakage)
    print("phương trình tính f_leakage=",Eta_InsScr)
    f_VentRoofSide=def_f_VentRoofSide(C_d,A_Flr,U_Roof,A_Roof,U_Side,A_Side,g,h_SideRoof,Delta_T_AirOut,T_mean,C_w,v_Wind)
    print("phương trình tính f_VentRoofSide=",f_VentRoofSide)
    ff_VentSide=def_f_VentRoofSide(C_d,A_Flr,U_Roof,0,U_Side,A_Side,g,h_SideRoof,Delta_T_AirOut,T_mean,C_w,v_Wind) # A_Roof=0
    print("phương trình tính ff_VentSide=",ff_VentSide)
    f_ventSide = def_f_VentSide(Eta_InsScr,ff_VentSide,f_leakage,U_ThScr,f_VentRoofSide,Eta_Side,Eta_SideThr)
    print("phương trình tính f_ventSide=",f_ventSide)
    f_ventForced = def_f_VentForced(Eta_InsScr,U_VentForced,Phi_VentForced,A_Flr)
    print("phương trình tính f_ventForced=",f_ventForced)
    MV_AirOut = def_MV_AirOut(M_water, f_ventSide, f_ventForced, VP_Air, VP_Out, R, T_Air, T_Out)
    print("phương trình tính MV_AirOut = ", MV_AirOut)
    print()
    
    print("Tính toán MV_TopOut")
    ff_VentRoof=def_ff_VentRoof(C_d,U_Roof,A_Roof,A_Flr,g,h_Roof,Delta_T_AirOut,T_mean,C_w,v_Wind)
    print("phương trình tính ff_VentRoof=",ff_VentRoof)
    f_ventRoof = def_f_VentRoof(Eta_InsScr,ff_VentRoof,f_leakage,U_ThScr,f_VentRoofSide,Eta_Roof,Eta_RoofThr)
    print("phương trình tính f_ventRoof=",f_ventRoof)
    MV_TopOut = def_MV_TopOut(M_water, f_ventRoof, VP_Top, VP_Out, R, T_Top, T_Out)
    print("phương trình tính MV_TopOut = ", MV_TopOut)
    print()
    
    dx1 = (MV_CanAir + MV_PadAir + MV_BlowAir + MV_FogAir - MV_AirThScr - MV_AirTop - MV_AirOut - MV_AirOut_Pad - MV_AirMech)/cap_VP_Air
    dx2 = (MV_AirTop - MV_TopCov_in - MV_TopOut)/cap_VP_Top
    print("Kết quả dx")
    print("VP'Air = ", dx1)
    print("VP'Top = ", dx2)
    return [dx1, dx2]

CÁC BIẾN THAY ĐỔI SAU MỖI VÒNG LẶP, BAN ĐẦU KHỞI TẠO LÀ 0.



In [6]:
T_Air=0
T_Top=0
T_Can=0
T_ThScr=0
T_MechCool=0
T_Cov_in=0
T_Out=0
I_Global=0
v_Wind=0
Eta_Roof=0
Eta_Side=0
VP_Air=0
VP_Top=0
VP_Can=0
VP_ThScr=0
VP_MechCool=0
VP_Cov_in=0
VP_Out=0
CO2_Air=0
cap_VP_Air=0
cap_VP_Top=0
MV_CanAir=0
MV_PadAir=0
MV_FogAir=0
MV_AirThScr=0
MV_AirOut=0
MV_AirOut_Pad=0
MV_AirTech=0
MV_AirTop=0
MV_TopCov_in=0
MV_TopOut=0


Lấy dữ liệu

In [7]:
DATA1=pd.read_csv("https://raw.githubusercontent.com/CEAOD/Data/master/GH_Cucumber/AutonomousGreenhouseChallengeFirstEdition(2018)/Sonoma/Greenhouse_climate.csv")
DATA2=pd.read_csv("https://raw.githubusercontent.com/CEAOD/Data/master/GH_Cucumber/AutonomousGreenhouseChallengeFirstEdition(2018)/meteo.csv")
df1=pd.DataFrame(DATA1)
df2=pd.DataFrame(DATA2)

Lấy dữ liệu cần thiết trong data



In [8]:
df=pd.DataFrame()
df[['GHtime','CO2air','Tair','VentLee','Ventwind','RHair']]=df1[['GHtime','CO2air','Tair','VentLee','Ventwind','RHair']]
df[['Windsp','Iglob']]=df2[['Windsp','Iglob']]
df['GHtime']=pd.TimedeltaIndex(df['GHtime'],unit='d')+dt.datetime(1899,12,30)
#df
#df=df.dropna()
(df['GHtime'][1]-df['GHtime'][0]).total_seconds()
#for i in range(3,len(df['GHtime'])):
#    if ((df['GHtime'][i]-df['GHtime'][i-1]).total_seconds()>500)==True:
#        print(i)
df.dropna(inplace=True)
df.reset_index(drop=True,inplace=True)
type((df['GHtime'][2]-df['GHtime'][1]).total_seconds())
print(df)

                             GHtime       CO2air  Tair    VentLee  Ventwind  \
0     2018-08-14 00:10:00.000048000   427.000000  19.9  52.900000       2.9   
1     2018-08-14 00:15:00.000028799   443.000000  19.9  52.900000       2.9   
2     2018-08-14 00:20:00.000009600   443.999999  19.9  51.099999       2.9   
3     2018-08-14 00:24:59.999990399   435.000001  19.7  42.800001       2.9   
4     2018-08-14 00:29:59.999971200   442.000000  19.9  42.800000       2.9   
...                             ...          ...   ...        ...       ...   
32984 2018-12-07 00:34:59.999952000   952.999999  24.5   0.000000       0.0   
32985 2018-12-07 00:40:00.000019200   900.000004  24.5   0.000000       0.0   
32986 2018-12-07 00:45:00.000000000   965.000000  24.7   0.000000       0.0   
32987 2018-12-07 00:49:59.999980799  1169.999989  24.6   0.000000       0.0   
32988 2018-12-07 00:55:00.000048000  1001.000023  24.7   0.000000       0.0   

       RHair  Windsp  Iglob  
0       80.4     3.2 

CHẠY SOLVER EULER

In [9]:
#Lấy giá trị VP tại thời điểm bất kỳ sau khi đã loại bỏ các hàng có giá trị NaN (dropna)))
#Dòng lấy giá trị
Start=8276
#Lấy giá trị ban đầu đưa vào ListOfResult
RH_Air=float(df['RHair'][Start])/100
RH_Top=RH_Air
T_Air=float(df['Tair'][Start])
T_Top=T_Air+1
VP_Air=Calculate_vapour_pressure_From_Humdity(RH_Air,T_Air)
VP_Top=Calculate_vapour_pressure_From_Humdity(RH_Top,T_Top)
Eta_Side=float(df['Ventwind'][Start]/100)
Eta_Roof=float(df['VentLee'][Start]/100)
v_Wind=float(df['Windsp'][Start])
I_Global=float(df['Iglob'][Start])
#Giả định
T_Can=T_Air+1
T_Top=T_Air+1
T_MechCool=T_Air+1
T_Cov_in=T_Cov_in+1
T_Out=T_Air+5
T_ThScr=T_Air+1
#Tính các giá trị VP khác
VP_Can=Calculate_Saturation_vapour_pressure(T_Can)
VP_Cov_in=Calculate_Saturation_vapour_pressure(T_Cov_in)
VP_MechCool=Calculate_Saturation_vapour_pressure(T_MechCool)
VP_ThScr=Calculate_Saturation_vapour_pressure(T_ThScr)
VP_Out=Calculate_Saturation_vapour_pressure(T_Out)   
#Lấy giá trị của CO2 theo Data
CO2_Air=float(df['CO2air'][Start])
CO2_Air=float(ConvertAirFrom_PPM_To_mgDivm3(CO2_Air))
#Tính giá trị hàm dx
res_dx=dx(cap_VP_Air,cap_VP_Top, MV_CanAir, MV_PadAir, MV_FogAir, MV_AirThScr, MV_AirOut, MV_AirOut_Pad, MV_AirTech, MV_AirTop, MV_TopCov_in, MV_TopOut)


Tính toán cap_VP
phương trình tính cap_VPAir =  2.7649426454907136e-05
phương trình tính cap_VP_Top =  3.045753134444499e-05

Tính toán MV_CanAir
phương trình tính R_Can= 0.0
phương trình tính rf_R_Can= 7.962962962962962
phương trình tính S_r_S= 0.0066928509242848554
phương trình tính c_evap3= 4.093565442453595e-09
phương trình tính c_evap4= 5.193976434168143e-06
phương trình tính rf_CO2_Air_ppm= 1.0013487315489682
phương trình tính rf_VP_VP_Air= 4.120878827967275
phương trình tính r_s= 2694.410391083454
phương trình tính VEC_CanAir= 1.5040777768720146e-08
phương trình tính Mv_CanAir =  1.1658931764985942e-05

Tính toán MV_PadAir
phương trình tính f_Pad= 0.002542857142857143
phương trình tính MV_PadAir =  0.0

Tính toán MV_AirOut_Pad
phương trình tính MV_AirOut_Pad =  4.529569452595779e-05

Tính toán MV_FogAir
phương trình tính MV_FogAir =  0.0

Tính toán MV_BlowAir
phương trình tính MV_BlowAir =  1.5821428571428572e-09

Tính toán MV_AirThScr
phương trình tính HEC_AirThScr= 1.275
phươn